**Learn the Vocabulary**

We use the Multi30k dataset for english - german translation. This notebook uses the byte-pair encoding to fit a joint vocabulary for both the English and German language based on the most frequent byte pairs.

In [1]:
from torchtext.datasets import multi30k, Multi30k

multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"
multi30k.URL["test"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"

data = Multi30k()

In [2]:
train_data, val_data, test_data = data

In [3]:
train_data = list(train_data)

In [4]:
train_data[0]

('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'Two young, White males are outside near many bushes.')

In [5]:
def get_joint_text_corpus(data):
    joint_text = []
    for pair in data:
        joint_text.append(pair[0])
        joint_text.append(pair[1])
    return joint_text

text_joint_en_de = get_joint_text_corpus(train_data)
text_joint_en_de[:10]

['Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'Two young, White males are outside near many bushes.',
 'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.',
 'Several men in hard hats are operating a giant pulley system.',
 'Ein kleines Mädchen klettert in ein Spielhaus aus Holz.',
 'A little girl climbing into a wooden playhouse.',
 'Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.',
 'A man in a blue shirt is standing on a ladder cleaning a window.',
 'Zwei Männer stehen am Herd und bereiten Essen zu.',
 'Two men are at the stove preparing food.']

**How do we turn a text corpus into a fixed length vector?**

With the Byte-Pair Encoding. Let's implement it.

In [6]:
def extract_corpus_from_data(data, language, n_sentences):
    corpus = []
    for i in range(n_sentences):
        corpus.append(data[i][0 if language == "german" else 1])
    return corpus

N_SENTENCES = 1000
english_corpus = extract_corpus_from_data(train_data, "english", N_SENTENCES)
german_corpus = extract_corpus_from_data(train_data, "german", N_SENTENCES)

In [7]:
from byte_pair_encoder import BytePairEncoder
import timeit
from pathlib import Path

data_path = "data/universal_bpe_encoder.pkl"
if not Path(data_path).exists():
    encoder_universal = BytePairEncoder("universal", max_vocab_size=1000, use_start_token=True, use_end_token=True, use_padding_token=True, max_token_len=300)
    time = timeit.timeit(lambda: encoder_universal.learn_vocabulary_from_corpus(text_joint_en_de, n_processes=16), number=1)
    print(f"Time to learn vocabulary for universal BPE: {time:.2f} seconds")
    encoder_universal.save_vocabulary(data_path)
else:
    encoder_universal = BytePairEncoder("universal", max_vocab_size=1000, use_start_token=True, use_end_token=True, use_padding_token=True, max_token_len=300, load_vocabulary_from=data_path)
    encoder_universal._max_token_len = 300

100%|██████████| 893/893 [10:19<00:00,  1.44it/s]

Time to learn vocabulary for universal BPE: 619.11 seconds
[INFO] Saving vocabulary data to: data/universal_bpe_encoder.pkl


In [8]:
from torch.utils.data import DataLoader

dl_train = DataLoader(data[0], shuffle=True, batch_size=4)

In [9]:
sample = next(iter(dl_train))
input, labels = sample
input = encoder_universal.encode_corpus(list(input))
labels = encoder_universal.encode_corpus(list(labels))

/home/wac3st/Development/code/transformer/.venv/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:297: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [10]:
input_dec = encoder_universal.decode_corpus(input)
labels_dec = encoder_universal.decode_corpus(labels)
print(input_dec)
print(labels_dec)

['𝄆Eine kleine Band tritt vor Menschen in einem Nachtclub auf.𝄇♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪', '𝄆Ein Junge hält sich an einer Anlegestelle an einem blauen Griff.𝄇♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪', '𝄆Das kleine Kind steht in einer Kampfsportstellung.𝄇♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪♪